In [ ]:
#Preprocess

# Pre

In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json



USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")


from google.colab import drive
drive.mount('/content/gdrive')

corpus_name = "cornell"
corpus = os.path.join("/content/gdrive/My Drive/data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "utterances.jsonl"))

Mounted at /content/gdrive
b'{"id": "L1045", "conversation_id": "L1044", "text": "They do not!", "speaker": "u0", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "not", "tag": "RB", "dep": "neg", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": "L1044", "timestamp": null, "vectors": []}\n'
b'{"id": "L1044", "conversation_id": "L1044", "text": "They do to!", "speaker": "u2", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "to", "tag": "TO", "dep": "dobj", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": null, "timestamp": null, "vectors": []}\n'
b'{"id": "L985", "conversation_id": "L984", "text": "I hope so.",

In [2]:
#Helper Function class 
  #We will set the length of sentence that we will consider
MAX_LENGTH = 10  # Maximum sentence length to consider
class SetVocab:
  def __init__(self,vocab,corpus, corpus_name, datafile):
    self.vocab = vocab
    self.corpus = corpus
    self.datafile = datafile
    self.corpus_name = corpus_name

  def unicodeToAscii(self,s):
      return ''.join(
          c for c in unicodedata.normalize('NFD', s)
          if unicodedata.category(c) != 'Mn'
      )

  # Lowercase, trim, and remove non-letter characters
  def normalizeString(self,s):
      s = self.unicodeToAscii(s.lower().strip())
      s = re.sub(r"([.!?])", r" \1", s)
      s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
      s = re.sub(r"\s+", r" ", s).strip()
      return s

  # Read query/response pairs and return a voc object
  def readVocs(self):
      print("Reading lines...")
      # Read the file and split into lines
      lines = open(self.datafile, encoding='utf-8').\
          read().strip().split('\n')
      # Split every line into pairs and normalize
      pairs = [[self.normalizeString(s) for s in l.split('\t')] for l in lines]
      voc = self.vocab(self.corpus_name)
      return voc, pairs

  # Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
  def filterPair(self,p):
      # Input sequences need to preserve the last word for EOS token
      return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

  # Filter pairs using filterPair condition
  def filterPairs(self,pairs):
      return [pair for pair in pairs if self.filterPair(pair)]




In [3]:
# Default word tokens
UNK_token = 0
PAD_token = 1  # Used for padding short sentences
SOS_token = 2  # Start-of-sentence token
EOS_token = 3  # End-of-sentence token
# Create default tokens, these will be used to pad or signal


#Build a chatbot vocabulary based on the word corpus that we have 
class ChatbotVocab:
    def __init__(self, name):
        self.name = name
        # we will define later
        self.trimmed = False
        #create dictionaries to store the index and the count for each word in the corpus
        self.maptoindex = {}
        self.maptocount = {}
        #map index to word for faster retrieval
        self.index2word = {UNK_token: "UNK",PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD, consider unique only
        self.trimmed = False

    #any time we pass a sentence to the vocah
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    #called by the sentence for each word it hass
    def addWord(self, word):
      """
      Check if the word is in the corpus
      If not create new identity for it
      create an index for it, set count
      Increment the number of words in vocab
      """
      if word not in self.maptoindex:

          self.maptoindex[word] = self.num_words
          self.maptocount[word] = 1
          self.index2word[self.num_words] = word
          self.num_words += 1
      else:
          self.maptocount[word] += 1

    # Avoid noise by trimming certain words that are rare
    def trim(self, min_count=3):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.maptocount.items():
            if v >= min_count:
                keep_words.append(k)

        print('Keeping only these many words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.maptoindex), len(keep_words) / len(self.maptoindex)
        ))

        # Reset and add all non trimmed words again dictionaries
        self.maptoindex = {}
        self.maptocount = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)


def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.maptoindex:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.maptoindex:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed to {} pairs".format(len(keep_pairs)))
    return keep_pairs

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    #Create the Vocab Helper Class by 
    setup_voc = SetVocab(ChatbotVocab,corpus, corpus_name,datafile)
    voc, pairs = setup_voc.readVocs()
    pairs = setup_voc.filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    MIN_COUNT = 3    # Minimum word count threshold for trimming
    #trim the pairs
    pairs = trimRareWords(voc, pairs,MIN_COUNT)
    return voc, pairs

# Setup FilePath
# Upload the file in data/cornell/ in gooogle drive
datafile = os.path.join(corpus, "formatted_movie_lines.txt")
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
print(f'Words in our courpus {voc.num_words}')
print(f'Number of pairs {len(pairs)}')
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:5]:
    print(pair)


Start preparing training data ...
Reading lines...
Trimmed to 64313 sentence pairs
Counting words...
Keeping only these many words 7833 / 18079 = 0.4333
Trimmed to 53131 pairs
Words in our courpus 7836
Number of pairs 53131

pairs:
['they do to !', 'they do not !']
['she okay ?', 'i hope so .']
['wow', 'let s go .']
['what good stuff ?', 'the real you .']
['do you listen to this crap ?', 'what crap ?']


In [4]:
print(voc.num_words)

7836


In [5]:
# creating a batch from the sequences
def sent2index(voc, sentence):
    #return the index of each word in the corpus
    return [voc.maptoindex[word] for word in sentence.split(' ')] + [EOS_token]


def Padding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMask(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputBatch(l, voc):
    indexes_batch = [sent2index(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = Padding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputBathc(l, voc):
    indexes_batch = [sent2index(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = Padding(indexes_batch)
    mask = binaryMask(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputBatch(input_batch, voc)
    output, mask, max_target_len = outputBathc(output_batch, voc)
    return inp, lengths, output, mask, max_target_len



# Transformer

In [6]:
# Define Transformer
#helpers
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    # Define special symbols and indices
    UNK_token, PAD_token, SOS_token, EOS_token =  0,1, 2, 3
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_token).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_token).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [7]:
#helpers
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    # Define special symbols and indices
    UNK_token, PAD_token, SOS_token, EOS_token =  0,1, 2, 3
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_token).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_token).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [22]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
        
    def translate(self, src: Tensor, src_mask: Tensor, max_length: int) -> Tensor:
        """Translate the given source sequence into target sequence."""
        enc_outputs = self.encode(src, src_mask)
        # Create a tensor to hold the predicted output sequence
        tgt = torch.zeros(1, 1).fill_(BOS_IDX).type_as(src)
        for i in range(max_length):
            tgt_mask = (generate_square_subsequent_mask(tgt.size(0))
                        .type(torch.bool)).type_as(src_mask)
            # Predict the next word in the target sequence
            with torch.no_grad():
                output = self.decode(tgt, enc_outputs, tgt_mask)
                pred = self.generator(output[:, -1])
            # Append the predicted word to the output sequence
            _, next_word = torch.max(pred, dim=1)
            next_word = next_word.item()
            tgt = torch.cat([tgt, torch.zeros(1, 1).fill_(next_word).type_as(src)], dim=0)
            # If the predicted word is the end-of-sentence token, stop generating
            if next_word == EOS_IDX:
                break
        # Remove the start-of-sentence and end-of-sentence tokens from the output sequence
        return tgt[1:-1]
    

In [9]:
#Create Custom Dataset and Dataloader
pairs[1][1]

'i hope so .'

# Create Custom Dataset and Dataloader

In [10]:
# split the data into trian and validate
validation_set_size = 0.2 
dataset_size = len(pairs)
validation_size = int(validation_set_size * dataset_size)
train_size = dataset_size - validation_size

In [11]:
from torch.utils.data import DataLoader, random_split


import torch
#Create a Dataloader
class Dataset_pairs(torch.utils.data.Dataset):
  'Make a dataset from the already existing pairs'
  def __init__(self, pairs):
        'Initialization'
        self.pairs = pairs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.pairs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample

        pair = self.pairs[index]
        # Load data and get label
        

        return pair 

CustomData = Dataset_pairs(pairs)

#The pairs from dataloader will be sent to the collate function, we will 
#get the desired output from here
def collate_fn(batch):
    op = batch2TrainData(voc,batch)
    return op

# Split the dataset into training and validation subsets
train_dataset, val_dataset = random_split(CustomData, [train_size, validation_size])

# Define batch sizes for training and validation
batch_size = 64
# Create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn)

# Define Training

In [12]:
# Define Parameters
SRC_VOCAB_SIZE = voc.num_words
TGT_VOCAB_SIZE = voc.num_words
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)
UNK_token, PAD_token, SOS_token, EOS_token = 0, 1, 2, 3
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_token)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.001 )

In [13]:
# training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
#                   ]
# print('tr')
# print(training_batches)
# print('tn')
# b = [random.choice(pairs) for _ in range(5)]
# print(b)
# i = 0
# for batch in train_loader:
#   print(batch)
#   if(i==0):
#     break

In [14]:
#Define Epochs/
total_epochs = 1
batch_size = 64
for epoch in range(total_epochs):
  # training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
  #                   for _ in range(n_iteration)]
  
  #train
  transformer.train()
  print_counter=0
  print_mul=0
  for i,batch_pair in enumerate(train_loader):    
    if i == len(train_loader) - 1:
      continue  # Skip the last batch
    tot_loss = 0
    
    #keep input_variable and variable lenght
    # train_batch = training_batches[iter-1

    input_variable, lengths, target_variable, mask, max_target_len = batch_pair
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(input_variable, target_variable)
    logits = transformer(input_variable.to('cuda'), target_variable.to('cuda'), src_mask.to('cuda'), tgt_mask.to('cuda'),src_padding_mask.to('cuda'), tgt_padding_mask.to('cuda'), src_padding_mask.to('cuda'))
    optimizer.zero_grad()
    # Define special symbols and indices
    UNK_token, PAD_token, SOS_token, EOS_token = 0, 1, 2, 3
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_token)
    tgt_out = target_variable
    logits = logits.to(torch.float32).to('cuda')  # Convert logits to floating-point data type
    tgt_out = tgt_out.to(torch.int64).to('cuda')
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    loss.backward()
    optimizer.step()
    print_counter+=1
    tot_loss += loss.item()
    if(print_counter == 5):
      
      print("Epoch: {}| iterations complete: {} | loss at this iteration {}".format(epoch, print_mul*print_counter,loss))
      print_counter = 0
  print('Epoch Training done, Now validating')

  #validate
  transformer.eval()
  for batch_pair in val_loader:
    
    val_loss = 0
    #keep input_variable and variable lenght
    # train_batch = training_batches[iter-1

    input_variable, lengths, target_variable, mask, max_target_len = batch_pair
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(input_variable, target_variable)
    logits = transformer(input_variable.to('cuda'), target_variable.to('cuda'), src_mask.to('cuda'), tgt_mask.to('cuda'),src_padding_mask.to('cuda'), tgt_padding_mask.to('cuda'), src_padding_mask.to('cuda'))
    optimizer.zero_grad()
    # Define special symbols and indices
    UNK_token, PAD_token, SOS_token, EOS_token = 0, 1, 2, 3
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_token)
    tgt_out = target_variable
    logits = logits.to(torch.float32).to('cuda')  # Convert logits to floating-point data type
    tgt_out = tgt_out.to(torch.int64).to('cuda')
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    val_loss += loss.item()
    
  print("Epoch: {}| Percent complete: {:.1f}%| Average loss: {:.4f}| Val loss: {:.4f}".format(epoch, epoch / total_epochs * 100,tot_loss,val_loss))


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and src_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 0| iterations complete: 0 | loss at this iteration 7.742984294891357
Epoch: 0| iterations complete: 0 | loss at this iteration 7.290112018585205
Epoch: 0| iterations complete: 0 | loss at this iteration 6.928518295288086
Epoch: 0| iterations complete: 0 | loss at this iteration 6.553684711456299
Epoch: 0| iterations complete: 0 | loss at this iteration 6.023582458496094
Epoch: 0| iterations complete: 0 | loss at this iteration 5.99150276184082
Epoch: 0| iterations complete: 0 | loss at this iteration 5.6794562339782715
Epoch: 0| iterations complete: 0 | loss at this iteration 5.187742710113525
Epoch: 0| iterations complete: 0 | loss at this iteration 4.8398590087890625
Epoch: 0| iterations complete: 0 | loss at this iteration 4.667515277862549
Epoch: 0| iterations complete: 0 | loss at this iteration 4.285816669464111
Epoch: 0| iterations complete: 0 | loss at this iteration 4.307641506195068
Epoch: 0| iterations complete: 0 | loss at this iteration 3.9078054428100586
Epoch: 0| 

In [15]:
import torch
from google.colab import drive
drive.mount('/content/gdrive')
# Save the entire model
torch.save(transformer.state_dict, 'model.pth')
model_path = os.path.join("/content/gdrive/My Drive/data/model", 'transformer.pt')
model_path_state= os.path.join("/content/gdrive/My Drive/data/model_dict", 'transformer_state.pt')
# Save only the model state dictionary
torch.save(transformer.state_dict(), model_path_state)
torch.save(transformer,model_path )


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
